In [ ]:
# Demo rapide des indicateurs

Ce notebook montre comment:
- Calculer les indicateurs et bundles
- Construire des features multi-timeframes
- Aplatir et normaliser pour un modèle ML
- Lancer un mini backtest prix-only


In [ ]:
from src.ai.feature_engineering.indicators import compute_indicator_bundle
from src.ai.feature_engineering.mtf_features import compute_mtf_bundle
from src.ai.feature_engineering.feature_vector import flatten_indicator_bundle, apply_normalization, to_ordered_vector, FeatureSpec
from src.ai.feature_engineering.backtest_indicators import backtest_price_only, BacktestConfig

# Données factices minimalistes à remplacer par de vraies séries
symbol = "BTC"
platform_prices = {
    "binance": {"price": 50000, "bid": 49990, "ask": 50010, "volume": 1200, "timestamp": __import__("datetime").datetime.utcnow(), "source": "exchange"},
    "okx": {"price": 50020, "bid": 50005, "ask": 50025, "volume": 900, "timestamp": __import__("datetime").datetime.utcnow(), "source": "exchange"},
    "coingecko": {"price": 50005, "bid": 50000, "ask": 50010, "volume": 2000, "timestamp": __import__("datetime").datetime.utcnow(), "source": "data_source"}
}
price_history = [{"price": 50000 + i} for i in range(100)]
spread_series = [0.5 for _ in range(60)]

bundle = compute_indicator_bundle(symbol, platform_prices, "binance", "okx", spread_series, price_history)
mtf = compute_mtf_bundle(symbol, platform_prices, "binance", "okx", spread_series, price_history)

flat = flatten_indicator_bundle(bundle)
specs = [FeatureSpec(name="momentum", min_value=-0.1, max_value=0.1, center=0.0, scale=0.02)]
norm = apply_normalization(flat, specs)
names, vec = to_ordered_vector(norm)

print("Features:", list(flat.keys())[:5], "...")
print("Vector size:", len(vec))

# Backtest factice
cfg = BacktestConfig(entry_threshold=0.0, take_profit=0.005, stop_loss=0.005)
res = backtest_price_only(symbol, [platform_prices]*50, "binance", "okx", [price_history]*50, [spread_series]*50, cfg)
res
